In [67]:

%matplotlib inline
from ipywidgets import interact
import pandas as pd
import numpy as np
import logging
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('dark_background')
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_colwidth', -1)
from sklearn.preprocessing import FunctionTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.pipeline import Pipeline
import category_encoders as ce

In [30]:
df = pd.read_csv('train_features.csv')
print(df.shape)
df.describe()

(37745, 103)


,id,member_id,loan_amnt,funded_amnt,installment,annual_inc,url,desc,dti,delinq_2yrs,...,revol_bal_joint,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog
count,37745.000000,0.0,37745.000000,37745.000000,37745.000000,3.774500e+04,0.0,0.0,37653.000000,37745.000000,...,4738.000000,4738.000000,4738.000000,4738.000000,4639.000000,4738.000000,4738.000000,4738.000000,4738.000000,1668.000000
mean,33579.507193,NaN,14913.304411,14913.304411,444.702933,7.962921e+04,NaN,NaN,18.508652,0.236111,...,33654.305192,0.724567,1.736809,11.296117,55.261199,2.855424,12.897425,0.048333,0.073660,36.332134
std,13609.172414,NaN,10153.364841,10153.364841,293.571158,8.052291e+04,NaN,NaN,19.670753,0.749590,...,29585.734663,1.068437,1.867062,6.557512,26.967270,3.054911,8.512730,0.455754,0.330458,24.058143
min,10000.000000,NaN,1000.000000,1000.000000,30.120000,0.000000e+00,NaN,NaN,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,21824.000000,NaN,7000.000000,7000.000000,222.990000,4.700000e+04,NaN,NaN,10.320000,0.000000,...,13968.500000,0.000000,0.000000,7.000000,35.350000,1.000000,7.000000,0.000000,0.000000,15.000000
50%,33603.000000,NaN,12000.000000,12000.000000,361.500000,6.700000e+04,NaN,NaN,16.680000,0.000000,...,25657.000000,0.000000,1.000000,10.000000,57.300000,2.000000,11.000000,0.000000,0.000000,34.000000
75%,45357.000000,NaN,20000.000000,20000.000000,610.170000,9.500000e+04,NaN,NaN,23.930000,0.000000,...,43273.500000,1.000000,3.000000,15.000000,76.900000,4.000000,17.000000,0.000000,0.000000,57.000000
max,57181.000000,NaN,40000.000000,40000.000000,1587.230000,9.300000e+06,NaN,NaN,999.000000,19.000000,...,290237.000000,6.000000,15.000000,58.000000,163.900000,35.000000,75.000000,20.000000,7.000000,125.000000


In [110]:

def clean(dat: pd.DataFrame) -> pd.DataFrame: 
    ''' refactored Ryan H's wrangle function from lecture today into method chaining'''
    todrop = ['id', # id is random
              'member_id', # all null
              'url', # all null
              'desc', # all null
              'title', # duplicate of purpose
              'grade', # duplicate of sub_grade
              'emp_title', # getting re-engineered, cardinality too high
              'zip_code' # cardinality too high
             ]

    many_nulls = ['sec_app_mths_since_last_major_derog',
                  'sec_app_revol_util',
                  'sec_app_earliest_cr_line',
                  'sec_app_mort_acc',
                  'dti_joint',
                  'sec_app_collections_12_mths_ex_med',
                  'sec_app_chargeoff_within_12_mths',
                  'sec_app_num_rev_accts',
                  'sec_app_open_act_il',
                  'sec_app_open_acc',
                  'revol_bal_joint',
                  'annual_inc_joint',
                  'sec_app_inq_last_6mths',
                  'mths_since_last_record',
                  'mths_since_recent_bc_dlq',
                  'mths_since_last_major_derog',
                  'mths_since_recent_revol_delinq',
                  'mths_since_last_delinq',
                  'il_util',
                  'emp_length',
                  'mths_since_recent_inq',
                  'mo_sin_old_il_acct',
                  'mths_since_rcnt_il',
                  'num_tl_120dpd_2m',
                  'bc_util',
                  'percent_bc_gt_75',
                  'bc_open_to_buy',
                  'mths_since_recent_bc']
    
    def wrangle_sub_grade(x):
        '''Transform sub_grade from "A1" - "G5" to 1.1 - 7.5'''
        first_digit = ord(x[0]) - 64
        second_digit = int(x[1])
        return first_digit + second_digit/10
    
    assigns = {# sub_grade to ordinal
        **{'sub_grade': dat.sub_grade.apply(wrangle_sub_grade)}, # sub_grade to ordinal
        # Convert percentages from strings to floats
        **{name: dat[name].str.strip('%').astype(float) 
                 for name in ['int_rate', 'revol_util']}, # Convert percentages from strings to floats
        # Transform earliest_cr_line to an integer: how many days it's been open
        **{'earliest_cr_line': (pd.Timestamp.today() - \
                                  pd.to_datetime(dat.earliest_cr_line, infer_datetime_format=True)
                               ).dt.days},  
        # Create features for three employee titles: teacher, manager, owner
        **{'emp_title_'+name: dat.emp_title.str.contains(name, na=False) 
                              for name in ['teacher', 'manager', 'owner']},
        # Transform features with many nulls to binary flags
        **{name: dat[name].isnull() for name in many_nulls},
        # For features with few nulls, do mean imputation
        **{name: dat[name].fillna(dat[name].mean()) for name in dat.select_dtypes(include=['int', 'float', 'float64']).columns}
              }
    
    return (dat.assign(emp_title = dat.emp_title.str.lower())
               .assign(**assigns)
               
               #.assign(revol_util = dat.revol_util.fillna(dat.revol_util.mean()))
               .drop(todrop, axis=1))
    
df = clean(pd.read_csv('train_features.csv'))
pd.set_option('display.max_columns', 20000)

#df.revol_util = df.revol_util.fillna(df.revol_util.mean())

df.isna().sum().sum()

#df.assign(revol_util = df.revol_util.fillna(df.revol_util.mean())).isna().sum().sum()

#df.revol_util.mean()


'revol_util' in df.select_dtypes(include=['int', 'float', 'float64']).columns

Index(['loan_amnt', 'funded_amnt', 'int_rate', 'installment', 'sub_grade',
       'annual_inc', 'dti', 'delinq_2yrs', 'earliest_cr_line',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'annual_inc_joint', 'dti_joint', 'acc_now_delinq', 'tot_coll_amt',
       'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m',
       'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util',
       'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util',
       'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m',
       'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util',
       'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct',
       'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl',
       'mort_acc', 'mths_since_recent_bc', 'mths_since_recent_bc_dlq',
       'mths

In [39]:
pipe = Pipeline([('clean', FunctionTransformer(func=clean)),
                 ('encode', ce.BinaryEncoder()), 
                 ('classify', DecisionTreeClassifier(max_depth=5, class_weight='balanced'))])


X_train = pd.read_csv('train_features.csv')
X_test = pd.read_csv('test_features.csv')
y_train = pd.read_csv('train_labels.csv')['charged_off']
#y_test = pd.read_csv('test_labels.csv')['charged_off']
sample_submission = pd.read_csv('sample_submission.csv')


cross_val_score(pipe, X_train, y_train, cv=5, scoring='roc_auc')

/home/quinn/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_function_transformer.py:98: FutureWarning: The default validate=True will be replaced by validate=False in 0.22.
  "validate=False in 0.22.", FutureWarning)
/home/quinn/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:542: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


ValueError: could not convert string to float: ' 60 months'

In [27]:
!ls

LCDataDictionary.xlsx  sample_submission.csv  train_features.csv
loans.ipynb	       test_features.csv      train_labels.csv


loan_amnt                                 0
funded_amnt                               0
term                                      0
int_rate                                  0
installment                               0
sub_grade                                 0
emp_length                              181
home_ownership                            0
annual_inc                                0
purpose                                   0
addr_state                                0
dti                                       7
delinq_2yrs                               0
earliest_cr_line                          0
inq_last_6mths                            0
mths_since_last_delinq                 1104
mths_since_last_record                 1693
open_acc                                  0
pub_rec                                   0
revol_bal                                 0
revol_util                                6
total_acc                                 0
initial_list_status             